# Personal-Rank 公式推导

## 算法抽象

### 文字描述

对于用户 A 进行个性化推荐，从用户 A 节点开始在用户物品二分图 random walk，以 alpha 的概率从 A 的出边中等概率的选择一条游走过去，达到该顶点后（例如 a 顶点），有 alpha 的概率继续从顶点 a 的出边中等概率的选择一条继续游走到下一个节点，或者 (1 - aplha) 的概率回到起点 A，多次迭代。直到各顶点对用 A 的重要程度收敛。

### 数学公式

$$\begin{equation}
PR(v)=\left\{
\begin{aligned}
\alpha * \sum_{\hat v \in in(v)} \frac{PR(\hat v)}{out(\hat v)} ...(v \not = v_A) \\
(1- \alpha) + \alpha * \sum_{\hat v \in in(v)} \frac{PR(\hat v)}{out(\hat v)} ...(v = v_A)
\end{aligned}
\right.
\end{equation}$$

### 矩阵式

$$ r = (1-\alpha) r_0 + \alpha M^Tr $$

$$ M_{ij} = \frac{1}{out(i)}j \in out(i) else 0 $$

$$ (E - \alpha M^T) * r = (1 - \alpha) r_0 $$

$$ r = (E - \alpha M^T)^{-1} (1 - \alpha) r_0 $$